In [1]:
!pip install unsloth[colab-new]@git+https://github.com/unsloth/unsloth.git
!pip install --no-deps xformers "peft>=0.8.0" trl

  Cloning https://github.com/unsloth/unsloth.git to /tmp/pip-install-mdhy8lhc/unsloth_f4ebbbf01ad040b2bf3214c3fcb1721a
  Running command git clone --filter=blob:none --quiet https://github.com/unsloth/unsloth.git /tmp/pip-install-mdhy8lhc/unsloth_f4ebbbf01ad040b2bf3214c3fcb1721a
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/unsloth/unsloth.git /tmp/pip-install-mdhy8lhc/unsloth_f4ebbbf01ad040b2bf3214c3fcb1721a did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/unsloth/unsloth.git /tmp/pip-install-mdhy8lhc/unsloth_f4ebbbf01ad040b2bf3214c3fcb1721a did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from

In [2]:
!pip install unsloth "peft>=0.8.0" trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.7/348.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
      Successfully uninstalled torchao-0.10.0
  Attempting uninstall: pyarrow
    Found e

##Mistral

In [ ]:
from unsloth import FastLanguageModel
import torch

# 💡 Modello BASE (Non Instruct)
model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
max_seq_length = 2048

# Caricamento del Modello e del Tokenizer in 4-bit (QLoRA)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

# Configurazione LoRA (Identica per l'efficienza)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

==((====))==  Unsloth 2025.10.10: Fast Mistral patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from datasets import load_dataset
import ast

# 1. Carica il tuo dataset
# Sostituisci con il percorso del tuo file locale (.json, .csv) o un dataset HF
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/dataset_final-gemini2-5.csv", split = "train")

# 2. Funzione di formattazione per la Generazione Continua (CLM)
def formatting_function_clm(examples):
    # La chiave 'text' è il campo che il tokenizer affinerà
    texts = []
    instruction = "Trova la soluzione al seguente gioco: date in input 5 parole non legate tra loro, dimmi qual è la parola in comune che risulta essere legata ad ognuna delle 5 parole"

    # 📝 Esempio: assumi che i tuoi dati abbiano le colonne 'header' e 'content'
    for clues_str, explanation_str, solution in zip(examples["clue"], examples["explanation"], examples['solution']):
        # Convert the string representation of lists to actual lists
        clues = ast.literal_eval(clues_str)
        explanation = ast.literal_eval(explanation_str)

        # ➡️ Unisci i campi in un singolo flusso di testo che vuoi che il modello apprenda.
        # Usa un formato che separa logicaamente i documenti, ma senza prompt conversazionali.
        text = f"ISTRUZIONE: {instruction}\n LE CINQUE PAROLE: {(', '.join(clues).upper())}\n\n RAGIONAMENTO: {(', '.join(explanation).upper())} \nSOLUZIONE: {solution}\n\n<|endofturn|>"
        texts.append(text)

    return { "text" : texts, }

# Mappa la funzione al dataset
dataset = dataset.map(formatting_function_clm, batched=True,)

Map:   0%|          | 0/454 [00:00<?, ? examples/s]

In [ ]:
dataset[0]

{'title': "La Ghigliottina - L'EreditÃ\xa0 19/11/2024'",
 'uploader': 'Rai',
 'episode_day': '19/11/2024',
 'url': 'https://www.youtube.com/watch?v=Yq5jddSisew',
 'frame_namefile': '19-11-2024.jpg',
 'words': 'GIORNO,PASSARE,LIBRO,GIOCHI,REALTA,23.125',
 'result': '```json\n{\n  "clue": ["GIORNO", "PASSARE", "LIBRO", "GIOCHI", "REALTA"],\n  "explanation": [\n    "fatti del giorno (quelli raccontati nei programmi di attualità)",\n    "passare alle parole ai fatti (prendere una decisione e metterla in pratica)",\n    "libro dei fatti (riferimento al libro che racconta determinati avvenimenti, come il Guinness World Record)",\n    "a giochi fatti (nel momento in cui si è scritto)",\n    "realtà dei fatti (perché è una persona concreta che guarda alla realtà dei fatti)"\n  ],\n  "solution": "FATTI"\n}\n```',
 'clue': "['GIORNO', 'PASSARE', 'LIBRO', 'GIOCHI', 'REALTA']",
 'explanation': "['fatti del giorno (quelli raccontati nei programmi di attualità)', 'passare alle parole ai fatti (prend

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # ⬅️ Usa il campo di testo continuo
    max_seq_length = max_seq_length,
    dataset_num_workers = 4,
    packing = False,

    args = TrainingArguments(
        per_device_train_batch_size = 2,    # Basso per Colab T4
        gradient_accumulation_steps = 4,  # Simula un batch size di 8
        warmup_steps = 5,
        num_train_epochs = 6,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        report_to="none",
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
    ),
)

# AVVIA IL FINE-TUNING
trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/454 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 454 | Num Epochs = 6 | Total steps = 342
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.321500
2,2.481800
3,2.129300
4,1.945400
5,1.688200
6,1.502500
7,1.413400
8,1.183100
9,1.138800
10,1.193200


In [ ]:
# Salva gli adapter LoRA
# model.save_pretrained_merged("mistral-base-finetuned-ghigliottinai2", tokenizer, save_method = "lora")
# 3. Usa il metodo nativo PEFT 'save_pretrained' sul modello
# Questo salva automaticamente solo i pesi LoRA aggiunti.

OUTPUT_DIR = "/content/drive/MyDrive/mistral_instruct_adapter/with-explanation-6epoch"
model.save_pretrained(OUTPUT_DIR)

# 4. Salva il Tokenizer separatamente (è comunque piccolo)
tokenizer.save_pretrained(OUTPUT_DIR)

('/content/drive/MyDrive/mistral_instruct_adapter/with-explanation-6epoch/tokenizer_config.json',
 '/content/drive/MyDrive/mistral_instruct_adapter/with-explanation-6epoch/special_tokens_map.json',
 '/content/drive/MyDrive/mistral_instruct_adapter/with-explanation-6epoch/chat_template.jinja',
 '/content/drive/MyDrive/mistral_instruct_adapter/with-explanation-6epoch/tokenizer.model',
 '/content/drive/MyDrive/mistral_instruct_adapter/with-explanation-6epoch/added_tokens.json',
 '/content/drive/MyDrive/mistral_instruct_adapter/with-explanation-6epoch/tokenizer.json')

In [7]:
def create_prompt(clues):
  instruction = "Trova la soluzione al seguente gioco: date in input 5 parole non legate tra loro, dimmi qual è la parola in comune che risulta essere legata ad ognuna delle 5 parole"
  text = f"{instruction}\n LE CINQUE PAROLE: {(", ".join(clues).upper())}\n"
  return text

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList
import torch

# ⚠️ DEFINISCI LA TUA SEQUENZA DI STOP ESATTA
STOP_SEQUENCE = "<|endofturn|>"
stop_token_ids = tokenizer.encode(STOP_SEQUENCE, add_special_tokens=False)
stop_token_ids = torch.tensor(stop_token_ids, device=model.device)
stop_len = len(stop_token_ids)

# --- CLASSE AGGIORNATA ---
class RobustStoppingCriteria(StoppingCriteria):
    def __init__(self, stop_token_ids, start_idx):
        self.stop_token_ids = stop_token_ids
        self.stop_len = len(stop_token_ids)
        self.start_idx = start_idx # Indice da cui iniziare a controllare

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # Controlla solo gli ultimi token generati
        current_sequence = input_ids[0]

        # Inizia a controllare solo dopo che il modello ha generato almeno la lunghezza del token di stop
        if current_sequence.shape[-1] < self.stop_len:
            return False

        # Controlla se la sequenza di stop si trova nell'ultima porzione dell'output
        # Controlliamo in una finestra che è grande quanto la lunghezza del token di stop

        # Controlla la corrispondenza esatta con gli ultimi token
        if torch.equal(current_sequence[-self.stop_len:], self.stop_token_ids):
            return True

        return False

# 3. Crea la lista dei criteri di stop
stopping_criteria = StoppingCriteriaList([RobustStoppingCriteria(stop_token_ids, inputs['input_ids'].shape[1])])

NameError: name 'inputs' is not defined

In [ ]:
from peft import PeftModel
from transformers import TextStreamer
import torch

# Per ricaricare il modello per l'inferenza (in un nuovo notebook)
# Ricarica il modello base (in 4-bit)
# base_model, _ = FastLanguageModel.from_pretrained(
#     model_name = model_name,
#     max_seq_length = max_seq_length,
#     load_in_4bit = True,
# )

# # Carica gli adapter LoRA e uniscili al modello base
# peft_model = PeftModel.from_pretrained(base_model, "mistral-base-finetuned-ghigliottinai")

FastLanguageModel.for_inference(model) # Prepara il modello per l'inferenza

prompt = create_prompt(clues=['maestro', 'fuori', 'cicale', 'rispondere', 'proteste'])
print(prompt)
# Tokenizzazione
inputs = tokenizer(
    [prompt],
    return_tensors = "pt",
).to("cuda")

# Generazione del testo (completamento)
streamer = TextStreamer(tokenizer, skip_prompt = True)

_ = model.generate(
    **inputs,
    streamer = streamer,
    max_new_tokens = 256,
    use_cache = True,
    #stopping_criteria = stopping_criteria
)

Pensa passo dopo passo usando la chain of thoughts 
ISTRUZIONE: Trova la soluzione al seguente gioco: date in input 5 parole non legate tra loro, dimmi qual è la parola in comune che risulta essere legata ad ognuna delle 5 parole
 LE CINQUE PAROLE: MAESTRO, FUORI, CICALE, RISPONDERE, PROTESTE


RAGIONAMENTO: MAESTRO DEL VIOLINO (RIFERIMENTO A PINOCCHIO), FUORI FORMA (RIFERIMENTO A PINOCCHIO E ALLE PROTESTE), CICALE E IL VIOLINO (RIFERIMENTO A 'CICALE DEL VIOLINO'), RISPONDERE COL VIOLINO (RIFERIMENTO A SARAH McLACHLAN E AL SUO TALENTO), VIOLINO DELLE PROTESTE (RIFERIMENTO AI GIORNI DI CORPO LIBERO E AI PROTESTATORI) 
SOLUZIONE: VIOLINO

<|endofturn|></s>


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#caricamento da checkpoint precedente
from unsloth import FastLanguageModel
import torch
from peft import PeftModel

# ⚠️ ASSICURATI DI MONTARE GOOGLE DRIVE QUI se hai salvato su Drive
# from google.colab import drive; drive.mount('/content/drive')

# Ricarica i parametri base (devono essere IDENTICI al training)
max_seq_length = 2048
model_name = "unsloth/mistral-7b-bnb-4bit" # Usa lo stesso modello base!

# 1. Carica il modello base in 4-bit (questa è l'operazione da 15 GB)
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

# ⚠️ CAMBIA 'OUTPUT_DIR' con il PERCORSO ESATTO dove hai salvato l'adapter
OUTPUT_DIR = "/content/drive/MyDrive/mistral_base_adapter"

# 2. Carica l'adapter LoRA (solo i ~100 MB) e uniscilo al modello base
# PeftModel prende il modello base (base_model) e applica i pesi da OUTPUT_DIR
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

# 3. Prepara per l'inferenza (passaggio Unsloth per ottimizzare la generazione)
FastLanguageModel.for_inference(model)

print("✅ Modello base e Adapter LoRA caricati e uniti in memoria. Pronto per l'inferenza.")

## let's try with Gemma fine-tuned on reasoning


In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# 1. Carica il modello Gemma con QLoRA/4bit
# Usiamo Gemma-3 4B IT per essere veloci e accessibili in ambienti con VRAM limitata
max_seq_length = 2048 # Imposta la lunghezza massima della sequenza
dtype = None # Seleziona automaticamente bfloat16 o float16
load_in_4bit = True # Usa QLoRA per l'efficienza

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-3-4b-it", # o "gemma-7b-it" se hai più VRAM
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 2. Configura gli adattatori LoRA
# Allena solo i moduli di attenzione (standard LoRA)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rango LoRA. 16 è un buon valore di partenza.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Ottimizzazione per VRAM
    random_state = 42,
    use_flash_attention_2 = True, # Velocizza l'addestramento
    max_seq_length = max_seq_length,
)

# 3. Carica e formatta il Dataset GSM8K per CoT

# Carica la traccia di training di GSM8K
gsm8k_dataset = load_dataset("gsm8k", "main", split = "train")

# Formattazione del prompt per la CoT.
# Gemma IT usa <start_of_turn>user / <start_of_turn>model
def formatting_prompts_func(examples):
    # La chiave è inserire la Chain-of-Thought (CoT) prima della risposta finale
    # La colonna 'answer' in GSM8K contiene la CoT e la risposta finale

    formatted_texts = []

    # GSM8K ha la colonna 'question' e 'answer' (che include il ragionamento)
    for question, answer in zip(examples['question'], examples['answer']):
        # Formato Gemma Instruction-Tuned (IT)
        text = f"""<start_of_turn>user
{question}<end_of_turn>
<start_of_turn>model
{answer}<end_of_turn>"""
        formatted_texts.append(text)

    return { "text" : formatted_texts }

# Applica la formattazione
gsm8k_dataset = gsm8k_dataset.map(formatting_prompts_func, batched = True,)

# 4. Avvia l'Addestramento

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = gsm8k_dataset,
    dataset_text_field = "text", # Colonna di testo che contiene l'input e l'output CoT
    max_seq_length = max_seq_length,
    dataset_num_workers = 4,
    packing = True, # Pacchettizza più esempi per un uso efficiente della GPU
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 1, # Di solito 2-3 epoch sono sufficienti per CoT
        learning_rate = 5e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        report_to="none",
        output_dir = "gemma_4b_cot_finetune",
        optim = "adamw_8bit",
        seed = 42,
    ),
)

from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

trainer.train()

# 5. Salva il modello LoRA adattato
#model.save_pretrained_merged("gemma_cot_merged_model", tokenizer, save_method = "json")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


INFO 10-29 11:10:25 [__init__.py:216] Automatically detected platform cuda.
ERROR 10-29 11:10:27 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.11: Fast Gemma3 patching. Transformers: 4.57.1. vLLM: 0.11.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

...

Map (num_proc=6):   0%|          | 0/7473 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 935
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 32,788,480 of 4,332,867,952 (0.76% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.367700
20,0.220800
30,0.182600
40,0.131900
50,0.131900
60,0.138200
70,0.140100
80,0.144500
90,0.150800
100,0.157100


TrainOutput(global_step=935, training_loss=0.15180019796850847, metrics={'train_runtime': 7077.8352, 'train_samples_per_second': 1.056, 'train_steps_per_second': 0.132, 'total_flos': 3.770352105702e+16, 'train_loss': 0.15180019796850847, 'epoch': 1.0})

In [4]:
#facciamo il finetuning a partire dal dataset creato da me con la chain of thoughts della ghigliottina
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# 1. Carica il modello Gemma con QLoRA/4bit
# Usiamo Gemma-3 4B IT per essere veloci e accessibili in ambienti con VRAM limitata
max_seq_length = 2048 # Imposta la lunghezza massima della sequenza
dtype = None # Seleziona automaticamente bfloat16 o float16
load_in_4bit = True # Usa QLoRA per l'efficienza

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-3-4b-it", # Changed from Gemma 3 4B to Gemma 2 9B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

instruction = 'Trova la soluzione al seguente gioco: date in input 5 parole non legate tra loro, dimmi qual è la parola in comune che risulta essere legata ad ognuna delle 5 parole \nLE CINQUE PAROLE: '

# 2. Configura gli adattatori LoRA
# Allena solo i moduli di attenzione (standard LoRA)
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Rango LoRA. 16 è un buon valore di partenza.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0.1,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Ottimizzazione per VRAM
    random_state = 42,
    use_flash_attention_2 = True, # Velocizza l'addestramento
    max_seq_length = max_seq_length,
)

# Carica la traccia di training
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/ghigliottinAI/dataset_cot4.csv", split = "train")

# Formattazione del prompt per la CoT.
# Gemma IT usa <start_of_turn>user / <start_of_turn>model
def formatting_prompts_func(examples):
    # La chiave è inserire la Chain-of-Thought (CoT) prima della risposta finale
    # La colonna 'answer' in GSM8K contiene la CoT e la risposta finale

    formatted_texts = []

    # GSM8K ha la colonna 'question' e 'answer' (che include il ragionamento)
    for clues, cot in zip(examples['clue'], examples['cot']):
        # Formato Gemma Instruction-Tuned (IT)
        text = f"""<start_of_turn>user
{instruction}{clues}<end_of_turn>
<start_of_turn>model
{cot}<end_of_turn><eos>"""
        formatted_texts.append(text)
    return { "text" : formatted_texts }

# Applica la formattazione
dataset = dataset.map(formatting_prompts_func, batched = True,)

from datasets import Dataset

# Dividi il dataset
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

# Poi usa train_dataset e eval_dataset nel trainer


# 4. Avvia l'Addestramento
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_workers = 4,
    packing = False,  # Disabilitato per preservare la struttura del reasoning
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,  # Ridotto
        warmup_ratio = 0.1,
        num_train_epochs = 3,  # Ridotto drasticamente
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 5,  # Più frequente per monitorare meglio
        eval_strategy = "steps",  # Aggiungi evaluation
        eval_steps = 20,  # Valuta ogni 20 steps
        save_strategy = "steps",
        save_steps = 20,
        load_best_model_at_end = True,  # Carica il miglior modello
        metric_for_best_model = "loss",
        save_total_limit = 2,  # Mantieni solo i 2 migliori checkpoint
        report_to="none",
        output_dir = "gemma_4b_cot_finetune",
        optim = "adamw_8bit",
        weight_decay = 0.01,  # Aggiungi regolarizzazione
        seed = 42,
    ),
)


from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

trainer.train()

# 5. Salva il modello LoRA adattato
#model.save_pretrained_merged("gemma_cot_merged_model", tokenizer, save_method = "json")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.12: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 363 | Num Epochs = 3 | Total steps = 273
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 16,394,240 of 4,316,473,712 (0.38% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,0.429900,1.654859
40,0.357900,1.424909
60,0.341600,1.378992
80,0.325200,1.347237
100,0.325400,1.352144
120,0.344400,1.374788
140,0.346900,1.411070
160,0.351900,1.434475
180,0.369800,1.450255
200,0.351700,1.466689


Unsloth: Not an error, but Gemma3ForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=273, training_loss=0.3617619813143552, metrics={'train_runtime': 2941.8537, 'train_samples_per_second': 0.37, 'train_steps_per_second': 0.093, 'total_flos': 2.063889774880416e+16, 'train_loss': 0.3617619813143552, 'epoch': 3.0})

In [5]:
# Salva gli adapter LoRA
# model.save_pretrained_merged("mistral-base-finetuned-ghigliottinai2", tokenizer, save_method = "lora")
# 3. Usa il metodo nativo PEFT 'save_pretrained' sul modello
# Questo salva automaticamente solo i pesi LoRA aggiunti.

OUTPUT_DIR = "/content/drive/MyDrive/gemma/gemma-4b-ghigliottina-simplerCot-3epoch-tunedByClaude"
model.save_pretrained(OUTPUT_DIR)

# 4. Salva il Tokenizer separatamente (è comunque piccolo)
tokenizer.save_pretrained(OUTPUT_DIR)

['/content/drive/MyDrive/gemma/gemma-4b-ghigliottina-cot-3epoch-tipByClaude/processor_config.json']

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, TextStreamer
import torch
from unsloth import FastLanguageModel

# Per ricaricare il modello per l'inferenza (in un nuovo notebook)
# Ricarica il modello base (in 4-bit)
max_seq_length = 2048 # Imposta la lunghezza massima della sequenza
dtype = None # Seleziona automaticamente bfloat16 o float16
load_in_4bit = True # Usa QLoRA per l'efficienza

base_model, _ = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-3-4b-it",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

# Carica gli adapter LoRA e uniscili al modello base
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/gemma/gemma-4b-ghigliottina-cot")


FastLanguageModel.for_inference(model) # Prepara il modello per l'inferenza
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/gemma/gemma-4b-ghigliottina-cot")

==((====))==  Unsloth 2025.10.11: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


In [8]:
prompt = create_prompt(clues=['maestro', 'fuori', 'cicale', 'rispondere', 'proteste'])
# Tokenizzazione - Explicitly pass None for other possible multimodal inputs
inputs = tokenizer(
    text=[prompt], # Pass the input_text in a list and explicitly label it as text
    return_tensors="pt",
).to("cuda")

# Genera la risposta
outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)

# Stampa l'output
print(tokenizer.batch_decode(outputs)[0])

<bos>Trova la soluzione al seguente gioco: date in input 5 parole non legate tra loro, dimmi qual è la parola in comune che risulta essere legata ad ognuna delle 5 parole
 LE CINQUE PAROLE: MAESTRO, FUORI, CICALE, RISPONDERE, PROTESTE
 RICERCA DI SOLUZIONI:
 LE CINQUE PAROLE SONO: "MAESTRO", "FUORI", "CICALE", "RISPONDERE", "PROTESTE"
 La prima parola è MAESTRO. La prossima è la parola che si lega al contesto di MAESTRO. La prossima è la parola "CICALE"

RAGIONAMENTO:
1. LA PRIMA SOLUZIONE RAGGIUNTA È: Il MAESTRO DEVE UNIRE PAROLE COME [ES. ASTRONOMI].
MAESTRO [GRUPPO ASTRONOMI] :
   MAESTRO [GRUPPO ASTRONOMI] NON ESCE.
2. LA SECONDA SOLUZIONE RAGGIUNTA È: GRUPPO [ES. TRAMER] NON ESCE.
FUORI [GRUPPO TRAMER]: [ES. GRANDE TRAMER]
3. L'ESERCIZIO DEVE SOGLIARE [GRUPPO ESPLOSIVO].
4. SE [ESPLOSIVO GRUPPO] NON ESCE:
LE SEGUENTI SONO [GRUPPO ESPOSIVO].

RAGIONAMENTO DEGLI ESSERI [ESPERIMO]:
L'ESPLOSIVO GRUPPO DEVE SOGLIARE [ESPLOSIVO GRUPPO].
ESPLOSIVO GRUPPO DEVE [GRUPPO ESPOSIVO].

[GRUPPO 

In [ ]:
from peft import PeftModel
from transformers import TextStreamer
import torch

# Per ricaricare il modello per l'inferenza (in un nuovo notebook)
# Ricarica il modello base (in 4-bit)
# base_model, _ = FastLanguageModel.from_pretrained(
#     model_name = model_name,
#     max_seq_length = max_seq_length,
#     load_in_4bit = True,
# )

# # Carica gli adapter LoRA e uniscili al modello base
# peft_model = PeftModel.from_pretrained(base_model, "mistral-base-finetuned-ghigliottinai")

FastLanguageModel.for_inference(model) # Prepara il modello per l'inferenza

prompt = create_prompt(clues=['maestro', 'fuori', 'cicale', 'rispondere', 'proteste'])
print(prompt)
# Tokenizzazione
inputs = tokenizer(
    [prompt],
    return_tensors = "pt",
).to("cuda")

# Generazione del testo (completamento)
streamer = TextStreamer(tokenizer, skip_prompt = True)

_ = model.generate(
    **inputs,
    streamer = streamer,
    max_new_tokens = 256,
    use_cache = True,
    #stopping_criteria = stopping_criteria
)

Pensa passo dopo passo usando la chain of thoughts 
ISTRUZIONE: Trova la soluzione al seguente gioco: date in input 5 parole non legate tra loro, dimmi qual è la parola in comune che risulta essere legata ad ognuna delle 5 parole
 LE CINQUE PAROLE: MAESTRO, FUORI, CICALE, RISPONDERE, PROTESTE

 SOLUZIONE: MAESTRO = CICALE ( entrambi hanno la lettere MA)
 SOLUZIONE: FUORI = PROTESTE ( entrambi hanno la lettera O)
 SOLUZIONE: CICALE = RISPONDERE ( entrambi hanno la lettere CE)
 SOLUZIONE: RISPONDERE = MAESTRO
 SOLUZIONE: CICALE = PROTESTE
 SOLUZIONED: MAESTRO = RISPONDERE
 SOLUZIONED: CICALE = PROTESTE
 SOLUZIONED: RISPONDERE = MAESTRO
 SOLUZIONED: CICALE = PROTESTE
 SOLUZIONED: PROTESTE = RISPONDERE
 SOLUZIONED: MAESTRO = CICALE
 SOLUZIONED: MAESTRO = RISPONDERE
 SOLUZIONED: CICALE = PRISTO
 SOLUZIONED: PRISTO = RISPONDERE
 SOLUZIONED: RISPONDERE = MAESTRO
 SOLUZIONED: PRISTO =  MAESTRO
 SOLUZIONED:  MAESTRO = RISPONDERE
 SOLUZIONED:  PRISTO


Ho aggiunto l'uso esplicito della CoT in gemma-3 4B, vediamo come si comporta con la sola CoT.
In un secondo momento potrei provare a finetunare partendo da Gemma con reasoning sul dataset creato da me e vediamo come evolve

##Velvet